In [ ]:
# imports

import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
sys.path.append('../')
import plotting.plottools
importlib.reload(plotting.plottools)
from plotting.plottools import plot_histogram
import normalizing
importlib.reload(normalizing)
from normalizing import normalize_by_omsjson

In [ ]:
# get some example histograms

#fname = '../data/examples/ZeroBias-Run2023C-PromptReco-v1-DQMIO-PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1.parquet'
#fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXBarrel-clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_1.parquet'
fname = '../data/data/ZeroBias-Run2024B-PromptReco-v1-DQMIO-PixelPhase1-Phase1_MechanicalView-PXForward-clusters_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_1.parquet'
df = pd.read_parquet(fname)
xbins = df['Xbins'][0]
ybins = df['Ybins'][0]
df = df[ (df['fromrun']==379008) & (df['fromlumi']<10) ]
df.reset_index(inplace=True)
nhists = len(df)
hists = np.array([df['histo'][i].reshape(xbins,ybins) for i in range(nhists)])
runs = np.array([df['fromrun'][i] for i in range(nhists)])
lumis = np.array([df['fromlumi'][i] for i in range(nhists)])
print('Shape of hists array: {}'.format(hists.shape))
print('Runs: {}'.format(runs))
print('Lumis: {}'.format(lumis))

In [ ]:
# define and load a json file with OMS data

omsfile = 'omsdata/omsdata_Run2024B-v1.json'
with open(omsfile, 'r') as f:
    omsjson = json.load(f)
print(list(omsjson.keys()))

In [ ]:
# get OMS data for relevant lumisections

runmask = np.isin(omsjson['run_number'], runs)
lumimask = np.isin(omsjson['lumisection_number'], lumis)
mask = (runmask & lumimask)
print('Indices: {}'.format(mask.nonzero()[0]))
for key in omsjson.keys():
    values = np.array(omsjson[key])[mask]
    print('{}: {}'.format(key, values))

In [ ]:
# plot raw data

for i, hist in enumerate(hists):
    fig, ax = plot_histogram(hist, figsize=(8,8))
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)

In [ ]:
# normalize by attribute

normalized_hists = normalize_by_omsjson(hists, runs, lumis, omsjson, 'pileup')

In [ ]:
# plot normalized histograms

for i, hist in enumerate(normalized_hists):
    fig, ax = plot_histogram(hist, figsize=(8,8))
    ax.text(0.02, 1.02, 'Run: {}, lumi: {}, normalized by pileup'.format(runs[i], lumis[i]), transform=ax.transAxes, fontsize=12)